<a href="https://colab.research.google.com/github/ujinyaa/github_/blob/feat%2F%237-ml/home/develop/code/ELMoEMBEDDING_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 엘모 코드


In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [ ]:
#elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)


https://www.kaggle.com/uciml/sms-spam-collection-dataset

케글에서 스팸 메일 분류하는 데이터 다운로드 해서 사용


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:10]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [ ]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])
print("데이터 전처리 완료, 데이터 개수 출력 ", len(X_data))

데이터 전처리 완료, 데이터 개수 출력  5572


<ipython-input-7-5c1c358c7711>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['v1'] = data['v1'].replace(['ham','spam'],[0,1])


In [ ]:
X_data[:10]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had y

In [ ]:
print(y_data[:10])

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]


In [ ]:
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print("학습 데이터 0.8 : " , n_of_train)
print("테스트 데이터 0.2 : ",n_of_test)

학습 데이터 0.8 :  4457
테스트 데이터 0.2 :  1115


In [ ]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

for i in range (0,5):
  print(X_train[i])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though


def ELMoEmbedding(x):
    tensor_x = tf.convert_to_tensor(tf.squeeze(tf.cast(x, tf.string)))
    return elmo(tensor_x)

In [ ]:
# ELMo 임베딩 함수 정의
def ELMoEmbedding(x, vocab_size, embedding_dim=300, max_len=None):
    """
    입력 텍스트 데이터를 처리하여 학습 가능한 문맥 임베딩을 생성합니다.
    - x: 텍스트 데이터 리스트
    - vocab_size: 토큰화된 단어 사전 크기
    - embedding_dim: 단어 임베딩 차원 (기본값: 300)
    - max_len: 시퀀스의 최대 길이 (None이면 데이터 기준 자동 설정)
    """
    # 토크나이저 정의 및 데이터 처리
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    if not max_len:
        max_len = max(len(seq) for seq in sequences)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post')

    # 임베딩 레이어 정의
    input_text = Input(shape=(max_len,), dtype='int32')
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(input_text)

    # 양방향 LSTM으로 문맥 임베딩 생성
    contextual_embedding = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)
    embedding_model = Model(inputs=input_text, outputs=contextual_embedding)

    # 임베딩 반환
    embeddings = embedding_model.predict(padded)
    return embeddings, padded, max_len, tokenizer


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
max_len = max(len(seq) for seq in X_train_seq)  # 시퀀스의 최대 길이
padded_sequences = pad_sequences(X_train_seq, maxlen=max_len, padding='post')

# 모델 정의
input_layer = Input(shape=(max_len,), dtype='int32')  # 텍스트 입력
embedding_layer = Embedding(input_dim=vocab_size, output_dim=300, mask_zero=True)(input_layer)  # 단어 임베딩
contextual_embedding = Bidirectional(LSTM(128, return_sequences=False))(embedding_layer)  # 양방향 LSTM
hidden_layer = Dense(256, activation='relu')(contextual_embedding)  # 은닉층
output_layer = Dense(1, activation='sigmoid')(hidden_layer)  # 출력층

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 171)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 171, 300)       │        300,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 171)            │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 256)            │        439,296 │ embedding[0][0],       │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │         65,792 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            257 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 805,345 (3.07 MB)

 Trainable params: 805,345 (3.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 모델 학습
try:
    # 모델 학습
    model.fit(
    np.array(padded_sequences),  # 패딩된 입력 데이터
    np.array(y_train),           # 라벨
    epochs=5,                    # 에포크 수
    batch_size=2,                # 배치 크기
    validation_split=0.2         # 검증 데이터 비율
)
    print("모델 학습 완료")
except Exception as e:
    print("오류 발생:", e)


Epoch 1/5
1783/1783 ━━━━━━━━━━━━━━━━━━━━ 656s 366ms/step - accuracy: 0.9517 - loss: 0.1660 - val_accuracy: 0.9798 - val_loss: 0.0753
Epoch 2/5
1783/1783 ━━━━━━━━━━━━━━━━━━━━ 666s 356ms/step - accuracy: 0.9912 - loss: 0.0386 - val_accuracy: 0.9787 - val_loss: 0.0648
Epoch 3/5
1783/1783 ━━━━━━━━━━━━━━━━━━━━ 683s 357ms/step - accuracy: 0.9967 - loss: 0.0144 - val_accuracy: 0.9675 - val_loss: 0.2229
Epoch 4/5
1783/1783 ━━━━━━━━━━━━━━━━━━━━ 638s 358ms/step - accuracy: 0.9928 - loss: 0.0388 - val_accuracy: 0.9787 - val_loss: 0.0739
Epoch 5/5
1783/1783 ━━━━━━━━━━━━━━━━━━━━ 641s 360ms/step - accuracy: 0.9985 - loss: 0.0059 - val_accuracy: 0.9776 - val_loss: 0.0753
모델 학습 완료


In [ ]:
# 모델 저장
model.save("elmo_lstm_model.h5")
print(model)
print("모델 저장 완료")


NameError: name 'model' is not defined

In [ ]:
X_test = {
     "text" :  ["This is a spam message.",
           "This is a normal message.",
           "Click here to win free money!",
           "Meeting at 10 AM.",
           "This if for announcement of service  ."],
    "label" : [1,0,1,0,1]
}
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')
predictions = model.predict(test_padded)


NameError: name 'tokenizer' is not defined

In [ ]:
# 결과 출력
for i, text in enumerate(X_test):
    print(f"문장: {text}")
    print(f"예측 확률(스팸): {predictions[i][0]:.4f}")
    print(f"예측 결과: {'스팸' if predictions[i][0] > 0.5 else '정상'}")
    print("-" * 50)

NameError: name 'X_test' is not defined

In [ ]:
# 테스트 데이터 크기 확인 및 수정
test_size = len(predictions)  # predictions 배열의 크기
X_test_data = X_data[:test_size]  # 테스트 데이터 크기를 predictions 크기에 맞춤

# 결과 출력
for i, text in enumerate(X_test_data):
    print(f"문장: {text}")
    print(f"예측 확률(스팸): {predictions[i][0]:.4f}")
    print(f"예측 결과: {'스팸' if predictions[i][0] > 0.5 else '정상'}")
    print("-" * 50)


NameError: name 'predictions' is not defined

In [ ]:
for i, text in enumerate(X_data[:5]):
    print(f"문장: {text}")
    print(f"예측 확률(스팸): {predictions[i][0]:.4f}")
    print(f"예측 결과: {'스팸' if predictions[i][0] > 0.5 else '정상'}")
    print("-" * 50)

NameError: name 'X_data' is not defined

이메일 스팸 데이터를 ELMo를 이용해 입력 텍스트를 크기 고정 벡터로 변환하고

Y_test= [
    "im spam"
    "im not spam"
    "am i spam?"
    "am i not spam"
    "are we spa.?"
    #"나는 스팸이 아니야. spam임",
    #"나는 스팸이 맞아. 근데 아닐수도 있고 메롱",
    #"나는 스팸이 맞아. 쿠팡 망해라",
    #"나는 스팸이 아니야. 유튜브 숏츠 중독자임",
    #"나는 스팸이야. 리챔아님"
]


from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

try:
    # ELMo 임베딩 생성
    elmo_embeddings = ELMoEmbedding(tf.constant(Y_test))  # 임베딩 생성
    elmo_embeddings_np = elmo_embeddings.numpy()  # NumPy 배열로 변환

    # CSV 저장
    df = pd.DataFrame(elmo_embeddings_np)  # Pandas DataFrame으로 변환
    df.to_csv('elmo_embeddings.csv', index=False)  # CSV 파일로 저장
    print("임베딩이 'elmo_embeddings.csv'에 저장되었습니다.")
except Exception as e:
    print("오류 발생:", e)


# 모델 정의
input_text = Input(shape=(1,), dtype=tf.string)  # 텍스트 입력
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024,))(input_text)  # ELMo 임베딩 사용
hidden_layer = Dense(256, activation='relu')(embedding_layer)  # 은닉층
output_layer = Dense(1, activation='sigmoid')(hidden_layer)  # 출력층

# 모델 생성
model = Model(inputs=[input_text], outputs=[output_layer])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 모델 컴파일

# 모델 요약 출력
model.summary()



In [ ]:
try:
    # 모델 예측 (입력을 NumPy 배열로 변환, ELMo는 문자열 입력 필요)
    predictions = model.predict(np.array(X_test), batch_size=1)

    # 결과 출력
    for i, text in enumerate(X_test):
        print(f"문장: {text}")
        print(f"실제 라벨: {y_test[i]} - 예측 확률(스팸): {predictions[i][0]:.4f}")
        print(f"예측 라벨: {'스팸' if predictions[i][0] > 0.5 else '정상'}")
        print("-" * 50)
except Exception as e:
    print("오류 발생:", e)


오류 발생: Invalid dtype: str14400
